In [ ]:
import pandas as pd
import numpy as np

In [ ]:
pd.options.display.max_columns = 500

# Feature Egineering 
### - Create time from last fight
### - ordinal encode Max_round
### - HomeTown will be logitude latitude 
### - Vectorize Name might help 

In [ ]:
df = pd.read_csv('clean_fights')
df.head()

## Create time since last fight 
- group each fighter (need to group so that both corners are looked at for time line)
- order based on date
- use an array the size of the data set
- replace none values to the index of that fighter 
- do this for each corner

In [ ]:
date = pd.to_datetime(df.Date)
df.Date = date
date.iloc[0]

In [ ]:
R_group = df.groupby('R_ID')
# len(R_group.groups)
R_group.groups

In [ ]:
B_group = df.groupby('B_ID')
B_group.groups

In [ ]:
129 in R_group.groups

In [ ]:
R_group.indices[129]

In [ ]:
R_group.groups[129][0]

In [ ]:
R_delta = np.empty(df.shape[0], dtype=pd.Timestamp)
B_delta = np.empty(df.shape[0], dtype=pd.Timestamp)

- iterate through all fighter IDs
- Use ID to get the Red corner and Blue corners into variables
- Create a data frame for each fighter containing the date of the fight and whether which corner they were in and the index and 
- Iterate through the dataframe and create a time delta from last match column
- Add that to corresponding index and corresponding array

In [ ]:
# Create list of all IDs
IDs = pd.concat([df.R_ID, df.B_ID])
IDs = IDs.drop_duplicates()

In [70]:
for ID in IDs:
    temp_df = pd.DataFrame()
    
    if ID in R_group.groups:
        corner = ['r'] * len(R_group.groups[ID])
        date = df.iloc[R_group.groups[ID]]
        temp_df['corner'].append(corner)
        temp_df['Date'].append(date)
        temp_df.head()
#     if ID in R_group.groups:
#         B_index = B_group[ID]
#         print('yes')
    
    

KeyError: 'corner'

In [65]:
df.iloc[[1, 2, 3]]

,BPrev,BStreak,B_Age,B_Height,B_HomeTown,B_ID,B_Location,B_Name,B_Weight,Date,Event_ID,Fight_ID,Last_round,Max_round,RPrev,R_Age,R_Height,R_HomeTown,R_ID,R_Location,R_Name,R_Weight,winby,winner,avg_weight_lbs,weight_class
1,0,0,36.0,172.0,"Chicago, Illinois United States",1054,"Chicago, Illinois United States",Ricardo Lamas,65.0,2014-02-03,646,4589,5,5,0,32.0,170.0,Manaus Brazil,1052,Rio de Janeiro Brazil,Jose Aldo,65.0,DEC,red,143.0,Bantamweight
2,0,0,39.0,167.0,"Isla Vista , California USA",959,"Sacramento, California USA",Urijah Faber,61.0,2014-02-03,646,4590,1,5,0,31.0,167.0,Natal Brazil,1527,Rio de Janeiro Brazil,Renan Barao,61.0,KO/TKO,red,134.2,Flyweight
3,0,0,33.0,167.0,"San Diego, CA USA",1056,"San Diego, CA USA",Danny Martinez,56.0,2014-02-03,646,4605,3,3,0,37.0,160.0,"San Jose, California USA",1253,"Tucson, Arizona USA",Chris Cariaso,56.0,DEC,red,123.2,Strawwweight


## Vectorize names with word to vec
 - I don't think this will work because there's already an ID

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
import warnings 
  
warnings.filterwarnings(action = 'ignore') 
  
import gensim 
from gensim.models import Word2Vec

In [ ]:
names = df.head().B_Name.values

In [ ]:
names

In [ ]:
tokens = [word_tokenize(x) for x in names]
tokens

In [ ]:
[Word2Vec(x) for x in tokens]

## Use Geopy to add longitude and latitude based off HomeTown

In [ ]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent='pwalis', timeout=3)
location = geolocator.geocode("Brazil", addressdetails=True)


In [ ]:
one = location


In [ ]:
import time
start = time.process_time()
[geolocator.geocode(x) for x in df.R_HomeTown.head(30)]
print(time.process_time() - start)

In [ ]:
R_geo = []
i = 0
for x in df.R_HomeTown:
    i+=1
    print(i)
    geo_info = geolocator.geocode(x)
    R_geo.append(geo_info)
# honestly I just want a green square

A lot of places aren't getting recognized as a location I should look into how the api expects the input
Research how to handle different errors 
if statements based on the error, If it can't get the location info add an entry into the dict that has the hometown string

In [ ]:
from geopy.exc import GeocoderTimedOut
i = 0
R_geo = dict()
for x in df.R_HomeTown:
    i+=1 
    try:
        location = geolocator.geocode(x).raw
        R_geo.update({i:location})
    except Exception as e:
        print("Error: geocode failed on input %s with message %s"%(x, e))
        

In [ ]:
len(R_geo)

In [ ]:
i=0
while i<1162:
    i+=1
    if len(R_geo[i])<10:
        print(R_geo[i])
        

In [ ]:
R_geo.keys()